# __Assistance with choosing a location where to move to__

## __1.1. Introduction__

### __1.1.1 Main reason behind this project__

 I've spent past year living in Warsaw, Poland and in a couple of months I'll be moving to Prague, Czech Republic both because of a work. So I'm using this opportunity and making a project that will help me with picking up the right location where to move to. But I will make it so that it will also be useful for other people who would consider moving to Prague. Or it even might be used by someone who needs basic overview of areas in Prague.
  There are many things to consider when you are relocating yourself. But basically it all leads to finding an area where real estate renting values are acceptable and at the same time you can find there all the right venues and easily accessible public transport.

### __1.1.2 Prague - The Basics__

 Situated on the Vltava River, Prague is the capital and the largest city of the Czech Republic. It is fourteenth largest city in the European Union and home to about 1.3 million people, but its larger urban zone has approximately a population of nearly 2 million. The surface area of Prague is 496 square kilometres. There are 10 municipal areas which consist 22 administrative areas, 57 city districts and 112 cadastral territories. [1]
 Distances to other major/capital cities: Berlin 350 km, Vienna 330 km, Budapest 530 km, Krakow 540 km

## __1.2. Data__

### __1.2.1 Locations__

 I will start with collecting geo-locational information of 57 city districts, which will help me build choropleth map. I have found a geojson file which has all the necessary geometry values of polygons based on locational data.[2] This will be used as a main map on which everything will be build.
 Then I will use Nominatim libraries to get locations of each cadastral district, because those which already exists are corrupt or incomplete. This will help me to place clustered data on the right locations on the map.

### __1.2.2 Venues__

 I'll use Foursquare API [3] to get a selection of venues in each cadastral district based on previous geolocation data, which will be later on processed, generalized and summarized so it will be easier to use for clusters.


### __1.2.3 Rents and extras__

 Not all the interesting data can be found with FourSquare. To choose the right location I need to know an estimation of rents and I'm also curious about density of transportation [5] and of recycling/sorting waste [6] in all areas. To collect data about average rents in Prague I've found two separate web pages of several different real estate broker businesses who were collaborating on these data sets.[4]


## __2.1. Methodology__

### __2.1.1 The basic map__

 I've obtained a geojson file wich has has the geometry description and location of each city district on 'open.praha'[2]. But this file was quite large
and detailed, later on when loading polygons into the map the process would have taken forever. To make it faster and easier for folium I've symplified the whole json file thanks to mapshaper [7].
 To create the map with folium I've used locational data obtained from nominatim and onto it I've superimposed districts' polygons with folium.GeoJson.

### __2.1.2 Rents__

 All the data here were scraped using BeautifulSoup because there was no exist dataframe about rents at all. First set of data gives us vague information about average movements in rents per year in the municipal areas. The dataframe and bar graph show us that there are some differences in municipal areas, but the rent doesn't change that rapidly.

![alt text](DF_rents_prg10.png "Title")

![alt text](bar_graph_rents.png "Title")


The second set is based on 57 city districts so it is much more thorough and useful for implementation into our map. The average price range here is from 95 to 430 CZK per m². I've used branca libraries to set a step colormap with eight sections for different rents range. This was embedded into the map using folium.style_function and the different rents range and colors were added using branca.element [8].

![alt text](prg_rents.png "Title")


 The map shows us that the most expensive location to rent a flat is not suprisingly the old historical center of town, but on the other hand it shows us that there are a lot of districts with low rents and slightly more than half of all of them are reasonably priced.

### __2.1.3 Foursquare dataset__

 I've created dataframe consisting of each cadastral and its radius again thanks to scraping with BeautifulSoup [9] and using Nominatim to acquire cadastral's locations. Based on this DF I've used Foursquare API to collect top venues, limit was set for 200. The resault it came with was 4525, but there were just 350 uniques categories. 
 One-hot Encoding helped me with spliting venues into unique categories and after droping a few unnecessary columns and grouping venues by sum in each cadastral I ended up with 112 rows and 351 columns.

![alt text](onehot_df.png "Title")

 Then I needed to narrow down the amount of columns and to somehow generalize and group those unique categories so they would still make sence. So after inspecting them I decided to manualy divide them into more general groups and those are added back in DF. I ended with 14 columns of groups.

![alt text](onehot_grouped.png "Title")

### __2.1.4 Recycling and buses__

 I wanted to add two more specific categories in which the first one would illustrate density of available waste bins/containers for sorting/recycling in each cadastral and second would specify density of bus and tram stops, beacause in FourSquare DF the group Transportation interprets every possible kind. I have found online datasets for both of these categories where every single one stop or bin has its own geolocation. Then I've used osmnx libraries[10] to get boundaries of each cadastral district (polygons) and so I could assign each unit into correct area and sum them up. I added everything into main DF with informations from FourSquare.

### __2.1.5 Normalizing and clustering__

 The main dataframe consists of several generalized groups of venues and points of interests for each cadastral districts. All the values were quite stretched and so to normalize them I've used sklearn.MinMaxScaler, which returns a value between 0 and 1. This new dataframe was then processed with AffinityPropagation from sklearn. "This algorythm takes as input measures of similarity between pairs of data points, and simultaneously considers all data points as potential exemplars. Real-valued messages are exchanged between data points until a high-quality set of exemplars and corresponding clusters gradually emerges."[11] It clustered dataset into 16 groups where I looked for the top 7 features/generalized categories.

![alt text](cluster_feature.png "Title")

 This gave me basic description features of each cluster and so I could finaly create the main dataframe were I placed cadastral names, its locations, cluster group and five density based features.

![alt text](final_df.png "Title")

## __3.1 Completing and summarizing__

### __3.1.1 Map finalizing__

 The last step was to merge this dataframe with the main map so when someone is trying to pick one of neighborhoods he can lean on visualized rents and grouped categories. 
 I've looped through each record in the GeoDataFrame and used folium's IFrame for making descriptions for each cadastral and then pinning them down on the map using geolocationfrom DF and folium.add_child and because there would be to many pins on the map I grouped them using MarkerCluster.

![alt text](map_fin.png "Title")

![alt text](map_fin_2.png "Title")

### __3.1.1 Results and summary__

 By using the map we will see Prague divided by districts' borders but on the other hand sorted thanks to clusters. Each pin and each area should help us with making our choice or at least point us in which direction should we focus our effort, so we won't loose time by going through all the information all across Prague. The choices here are really individual and everything depends on everyone's preferences. Simply said, the map tells us the average costs of living and top five categories of venues in specific area.
 You have a lot of cash, want to party, have a quick snack and the next day relax at a cafe? Than 'Staré Město' is your pick. You're good on cash but would rather have a beer or a coffee and go relax to park, go for 'Smíchov'. Low on cash and moving here because of a work, than 'Chodov' can be the right pick.
 So the main reason why I wanted to do this was because I'll be relocating there in the next couple of weeks. And going through all those data and now seeing the whole picture, it really helps me to pick the right location... And now just if there will be available flat to rent.

### References:
[1] https://www.welcometoprague.eu/basic-information-about-prague 

[2] http://opendata.praha.eu/dataset/ipr-mestske_casti

[3] https://foursquare.com/

[4] https://realitymix.centrum.cz/statistika-nemovitosti/byty-pronajem-prumerna-cena-pronajmu-1m2-mesic.html
    http://www.cenovamapa.eu/
    
[5] http://opendata.iprpraha.cz/CUR/DOP/DOP_PID_ZASTAVKY_B/WGS_84/DOP_PID_ZASTAVKY_B.json

[6] http://opendata.iprpraha.cz/CUR/ZPK/ZPK_O_Kont_TOitem_b/S_JTSK/ZPK_O_Kont_TOitem_b.json

[7] https://mapshaper.org/

[8] https://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd

[9] https://cs.wikipedia.org/wiki/Seznam_katastr%C3%A1ln%C3%ADch_%C3%BAzem%C3%AD_Prahy_podle_po%C4%8Dtu_obyvatel

[10] https://github.com/gboeing/urban-data-science

[11] https://www.toptal.com/machine-learning/clustering-algorithms